In [1]:
import pandas as pd
import json
import os
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import numpy as np
from folium.plugins import TimeSliderChoropleth
from branca.colormap import linear
import HTA_helpers as h

In [11]:
#path = 'data/

In [4]:
def gen_times_list(frequency):
    #****frequency defaults to mins. Enter value as number
    freq = str(frequency) + 'Min'
    #freq = freq
    time_list = []
    for time in pd.date_range('2018-11-06 07:00:00', '2018-11-06 21:00:00', freq=freq):
        time_list.extend(pd.date_range(time, freq='1H', periods=1).strftime("%Y-%m-%d %H:%M:%S").tolist())
    return time_list

In [2]:
def reg_count(df,freq):
    df = df
    freq = freq
    intervals = h.gen_times_list(freq)

    def int_filter(df,i):
        df = df[(df.checkin_time > intervals[i]) & (df.checkin_time < intervals[i+1])]
        return df

    count = []
    t_index = []
    for i in range((len(intervals)-5)):
        df2 = int_filter(df,i)
        d2 = len(df2.ct1)
        count.append(d2)
        #print(d2)
        t_index.append(intervals[i+1])
    index = pd.DatetimeIndex(t_index)
    return pd.Series(count,index=index )


In [11]:
pathx = 'C:/Users/SIS Lab Admin/Desktop/Fall 2019/Python stuff/working_data/Check_In_All/RhodeIsland_Voter_Checkin_Roster_11.06.2018.csv'
dfx = pd.read_csv(pathx)
dfx.head()

df = pd.read_csv(pathx)
time_intervals_a = h.gen_times_list()
#precincts = list(df['precinct'].unique())
p_df = df[(df.precinct > 2800) &(df.precinct <2899)]
precincts = list(p_df['precinct'].unique())

interval_counts = []
for i in range(len(precincts)):
    df2 = df[(df.precinct == precincts[i]) ]   
    counts = []
    
    for j in range(len(time_intervals_a)-1):
        df3 = df2[(df2.checkin_time > time_intervals_a[j]) & (df2.checkin_time < time_intervals_a[j+1])]    
        counts.append(len(df3))
        
    interval_counts.append(counts)
    
df4 = pd.DataFrame(interval_counts,index = precincts)
df4.head()
df4.T
#df4.to_csv('C:/Users/SIS Lab Admin/Desktop/test_data.csv')

,2866,2876,2826,2874,2844,2824,2863,2842,2823,2851,...,2836,2804,2840,2843,2833,2859,2810,2834,2878,2867
0,198,87,142,179,188,272,128,32,168,220,...,3,72,62,90,61,76,25,40,4,11
1,127,73,126,113,139,156,68,26,135,112,...,0,81,47,43,47,27,11,31,13,15
2,82,55,107,88,96,137,52,13,99,97,...,0,78,46,28,65,16,4,23,2,9
3,58,52,99,111,79,156,65,12,101,70,...,5,61,40,23,40,11,10,27,4,9
4,97,115,166,165,147,244,76,22,160,91,...,3,51,29,47,80,30,5,48,9,12
5,86,84,102,102,88,160,55,4,133,72,...,0,28,19,34,46,20,13,15,5,12


In [12]:
pathg = './data/RI Precinct OBJECTIDs.csv'
df = pd.read_csv(pathg)
df.iloc[1]
df

,NAME,OBJECTID
0,101,1
1,102,0
2,103,4
3,104,2
4,105,3
5,201,13
6,202,6
7,203,5
8,204,10
9,205,9


In [4]:


path = 'data/CI_for_Prov_norm.csv'
df = pd.read_csv(path)
df.head()

,Unnamed: 0,OBJECTID,n1,n2,n3,n4,n5,n6
0,2801,219,0.321429,0.214286,0.214286,0.092857,0.085714,0.071429
1,2802,224,0.385870,0.244565,0.119565,0.125000,0.065217,0.059783
2,2803,227,0.393665,0.201810,0.092308,0.105882,0.120362,0.085973
3,2804,226,0.194070,0.218329,0.210243,0.164420,0.137466,0.075472
4,2805,225,0.339514,0.195013,0.140665,0.101662,0.122762,0.100384


,11/6/2018 7:00,11/6/2018 8:00,11/6/2018 9:00,11/6/2018 10:00,11/6/2018 11:00,11/6/2018 12:00,11/6/2018 13:00,11/6/2018 14:00,11/6/2018 15:00,11/6/2018 16:00,11/6/2018 17:00,11/6/2018 18:00,11/6/2018 19:00,11/6/2018 20:00,NaN,NaN
2801,0.535714,0.121429,0.092857,0.042857,0.050000,0.050000,0.035714,0.050000,0.021429,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
2802,0.630435,0.054348,0.065217,0.081522,0.043478,0.038043,0.027174,0.032609,0.027174,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
2803,0.595475,0.043439,0.048869,0.047059,0.058824,0.055204,0.065158,0.054299,0.031674,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
2804,0.412399,0.134771,0.075472,0.102426,0.061995,0.070081,0.067385,0.053908,0.021563,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
2805,0.534527,0.076726,0.063939,0.051790,0.049872,0.054348,0.068414,0.070332,0.030051,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
2806,0.578431,0.029412,0.039216,0.058824,0.039216,0.058824,0.068627,0.058824,0.068627,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
2807,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
2808,0.569838,0.062753,0.047571,0.050607,0.055668,0.062753,0.068826,0.052632,0.028340,0.001012,0.0,0.0,0.0,0.0,NaN,NaN
2809,0.531429,0.057959,0.051429,0.052245,0.047347,0.076735,0.076735,0.069388,0.036735,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
2810,0.529412,0.044118,0.014706,0.044118,0.102941,0.044118,0.029412,0.147059,0.044118,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
